# Housekeeping

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlite3

from jsonschema.benchmarks.const_vs_enum import value
from pandas.tseries.offsets import *
from datetime import timedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

today = dt.datetime.today().strftime('%Y-%m-%d')
today

'2025-12-17'

# Pull data from WRDS - December 17, 2025

In [4]:
import wrds
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\jessf\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [7]:
count = db.get_row_count('comp', 'funda')
count

922407

In [9]:
funda = db.raw_sql("SELECT * FROM comp.funda WHERE indfmt='INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C' AND at > 0 AND at IS NOT NULL AND fyear > 2000", date_cols=['datadate'])
names = db.get_table('comp', 'names', columns=['gvkey', 'sic', 'naics'])
names = names.rename(columns={'sic': 'sica'})
fundq = db.get_table('comp', 'fundq', columns=['gvkey', 'datadate', 'rdq'])
comphist = db.get_table('crsp', 'comphist', columns=['gvkey', 'hconm', 'hstate', 'hcik', 'hadd1', 'haddzip', 'hcity', 'hfic', 'hloc', 'hsic', 'hchgdt', 'hchgenddt'])
comphist = comphist.rename(columns={'gvkey': 'gvkey1'})
comphist['hchgdt'] = pd.to_datetime(comphist['hchgdt'], errors='coerce')
comphist['hchgenddt'] = pd.to_datetime(comphist['hchgenddt'], errors='coerce')
company = db.get_table('comp', 'company', columns=['gvkey', 'conm', 'state', 'cik', 'addzip', 'city', 'fic', 'loc', 'add1', 'sic'])
company = company.rename(columns={'gvkey': 'gvkey1'})

In [10]:
len(funda)

212378

In [11]:
#check for duplicates and missing CIK
funda.duplicated(subset=['gvkey', 'datadate']).sum()

np.int64(0)

In [12]:
funda['cik'].isna().sum()

np.int64(21538)

In [34]:
funda.to_pickle(f"../Data/raw/compustat_funda_raw_{today}.pkl")

# Prepare compustat data

Bring in sic and naics codes

In [13]:
a1 = pd.merge(funda, names, on='gvkey', how='left')
len(a1)

212378

Bring in earnings announcement date

In [14]:
fundq['datadate'] = pd.to_datetime(fundq['datadate'], errors='coerce')
a2 = pd.merge(a1, fundq, on=['gvkey', 'datadate'], how='left')
len(a2)

212659

In [15]:
a3 = a2.sort_values(by=['gvkey', 'fyear']).drop_duplicates(subset=['gvkey','fyear'], keep='first')
len(a3)

212378

Use historical sic, when available.

In [16]:
a3['sicm'] = a3.apply(lambda row: row['sica'] if pd.notnull(row['at']) and pd.isnull(row['sich']) else row['sich'], axis=1)

Create additional variables - sic variations, month, year

In [17]:
a3['sicm'] = pd.to_numeric(a3['sicm'], errors='coerce')
a3['sic2'] = a3['sicm'].floordiv(100)
a3['sic3'] = a3['sicm'].floordiv(10)

a3['datadate'] = pd.to_datetime(a3['datadate'], errors='coerce')
a3['year'] = a3['datadate'].dt.year
a3['month'] = a3['datadate'].dt.month

Clean things up

In [18]:
a3 = a3.drop(columns=['sica', 'sich'])
a3 = a3.rename(columns={'sicm': 'sic'})

Bring in location/historical data based on recommendations in Utke (2019): https://ssrn.com/abstract=3212035

In [19]:
conn = sqlite3.connect(':memory:')
a3.to_sql('a3', conn, index=False)
comphist.to_sql('comphist', conn, index=False)

query = """
    SELECT a.*, b.hloc, b.hstate, b.hcity, b.hadd1, b.haddzip, b.hcik, b.hconm
    FROM a3 AS a
    LEFT JOIN comphist AS b
    ON a.gvkey = b.gvkey1
    AND b.hchgdt <= a.datadate
    AND (b.hchgenddt > a.datadate OR b.hchgenddt IS NULL)
"""

a4 = pd.read_sql_query(query, conn, parse_dates=['datadate'])
conn.close()

len(a4)

212378

Create separate df with datadate of first (oldest) LOC code and the location itself

In [20]:
z1 = a4.dropna(subset='hloc')
z1 = z1.sort_values(by=['gvkey', 'datadate'])
z1['initial'] = z1.groupby('gvkey').cumcount()==0
z1 = z1[z1['initial']==True]

Backfill the data

In [21]:
conn = sqlite3.connect(':memory:')
a4.to_sql('a4', conn, index=False, if_exists='replace')
z1.to_sql('z1', conn, index=False, if_exists='replace')

query = """
    SELECT a.*, b.hloc AS initial_hloc, b.hstate AS initial_hstate, b.hcity AS initial_hcity, b.hadd1 AS initial_hadd1, b.haddzip AS initial_haddzip, b.hcik AS initial_hcik, b.hconm AS initial_hconm
    FROM a4 AS a
    LEFT JOIN z1 AS b
    ON a.gvkey = b.gvkey
    AND a.datadate < b.datadate
"""

a5 = pd.read_sql_query(query, conn, parse_dates=['datadate'])
conn.close()

len(a5)

212378

For all firms, fill in missing location data with initial location data

In [22]:
conn = sqlite3.connect(':memory:')
a5.to_sql('a5', conn, index=False, if_exists='replace')
company.to_sql('company', conn, index=False, if_exists='replace')

query = """
    SELECT a.*, b.loc, b.state, b.city, b.addzip, b.add1
    FROM a5 AS a
    LEFT JOIN company AS b
    ON a.gvkey = b.gvkey1
"""

a6 = pd.read_sql_query(query, conn, parse_dates=['datadate'])
conn.close()

len(a6)

212378

Combine location variable into 1 column based on the best data available

In [23]:
a6['loc_use'] = a6.apply(lambda row: row['hloc'] if pd.notnull(row['hloc']) else row['initial_hloc'] if pd.notnull(row['initial_hloc']) else row['loc'], axis=1)
a6['state_use'] = a6.apply(lambda row: row['hstate'] if pd.notnull(row['hstate']) else row['initial_hstate'] if pd.notnull(row['initial_hstate']) else row['state'], axis=1)
a6['city_use'] = a6.apply(lambda row: row['hcity'] if pd.notnull(row['hcity']) else row['initial_hcity'] if pd.notnull(row['initial_hcity']) else row['city'], axis=1)
a6['add1_use'] = a6.apply(lambda row: row['hadd1'] if pd.notnull(row['hadd1']) else row['initial_hadd1'] if pd.notnull(row['initial_hadd1']) else row['add1'], axis=1)
a6['addzip_use'] = a6.apply(lambda row: row['haddzip'] if pd.notnull(row['haddzip']) else row['initial_haddzip'] if pd.notnull(row['initial_haddzip']) else row['addzip'], axis=1)
a6['cik_use'] = a6.apply(lambda row: row['hcik'] if pd.notnull(row['hcik']) else row['initial_hcik'] if pd.notnull(row['initial_hcik']) else row['cik'], axis=1)
a6['conm_use'] = a6.apply(lambda row: row['hconm'] if pd.notnull(row['hconm']) else row['initial_hconm'] if pd.notnull(row['initial_hconm']) else row['conm'], axis=1)

#Drop unneeded columns and rename
a6 = a6.drop(columns=['hloc', 'hstate', 'hcity', 'hadd1', 'haddzip', 'hcik', 'hconm',
                      'initial_hloc', 'initial_hstate', 'initial_hcity', 'initial_hadd1', 'initial_haddzip', 'initial_hcik', 'initial_hconm'])

a6 = a6.rename(columns={'loc_use': 'loc', 'state_use': 'state', 'city_use': 'city', 'add1_use': 'add1', 'addzip_use': 'addzip', 'cik_use': 'cik', 'conm_use': 'conm'})

len(a6)

212378

Save data in temp file with date of extract -- 12/17/2025; 212,378 records

In [32]:
from pathlib import Path
Path("../Data/temp").mkdir(parents=True, exist_ok=True)

In [33]:
a6.to_pickle(f"../Data/temp/compustat_funda_prepped_{today}.pkl")

# Create additional compustat variables - start here unless doing a new WRDS pull

Import compustat file

In [35]:
b1 = pd.read_pickle("../Data/temp/compustat_funda_prepped_2025-12-17.pkl")
len(b1)

212378

Handle missing values for key variables

In [36]:
key_vars = ['dlc','dltt','rect','recch','invt','invch','recd','spi','rcp','gdwl','intan','ppent','drc','drlt','wdp','tfva','ivaeq','ivao','ivst','txtubend','rll','clg','clt','xidoc','txr','txpd']
for var in key_vars:
    b1[var] = b1[var].fillna(0)

Firm size and market value

In [60]:
b1['size'] = b1['at'].apply(lambda x: np.log(x) if x > 0 else np.nan)
b1['market_value'] = b1['csho'] * b1['prcc_f']
b1['market_value'] = b1['market_value'].apply(lambda x: x if x > 0 else np.nan)
b1['log_market_value'] = b1['market_value'].apply(lambda x: np.log(x) if x > 0 else np.nan)
b1['mtb'] = b1['market_value'] / b1['at']
b1['tobinq'] = (b1['at'] - b1['ceq'] + b1['market_value']) / b1['at']

Profitability and Financial Performance

In [39]:
b1['roa'] = b1['ib'] / b1['at']
b1['ocf'] = b1['oancf'] / b1['at']
b1['is_loss'] = b1['ib'].apply(lambda x: 1 if x < 0 else 0)
b1['revenue'] = b1['revt'].apply(lambda x: np.log(x) if x > 0 else np.nan)
b1['sale_at'] = b1['sale'] / b1['at']
b1['cash_flow'] = b1['oibdp'] - (b1['rect'] + b1['invt'] + b1['aco'] - b1['lco'] - b1['ap']) - b1['capx']
b1['has_fcf'] = (b1['cash_flow']>0).astype(int)
b1['sga'] = b1['xsga'] / b1['sale']

Leverage and Capital Structure

In [44]:
b1['leverage'] = (b1['dlc'] + b1['dltt']) / b1['at']
b1['debt'] = b1['dltt'] / b1['at']
b1['has_issuance'] = ((b1['dltis'] > 0) | b1['sstk'] > 0).astype(int)

Liquidity and Working Capital

In [42]:
b1['rect_at'] = b1['rect'] / b1['at']
b1['invt_at'] = b1['invt'] / b1['at']
b1['invrec'] = (b1['invt'] + b1['rect']) / b1['at']
b1['current_assets'] = b1['act'] / b1['at']
b1['quick_ratio'] = (b1['act'] - b1['invt']) / b1['lct']
b1['current_ratio'] = b1['act'] / b1['lct']
b1['working_capital'] = (b1['act'] - b1['lct']) / b1['at']

Intangibles and Asset Structure

In [43]:
b1['ppent_at'] = b1['ppent'] / b1['at']
b1['gdwl_at'] = b1['gdwl'] / b1['at']
b1['intan_at'] = b1['intan'] / b1['at']
b1['other_intan_at'] = (b1['intan'] - b1['gdwl'] / b1['at'])
b1['fv_at'] = b1['tfva'] / b1['at']

Earnings Management and Special Items

In [45]:
b1['has_spec_items'] = (b1['spi'] != 0).astype(int)
b1['has_restructuring'] = (b1['rcp'] != 0).astype(int)
b1['has_gw_impair'] = (b1['gdwlip'] != 0).astype(int)
b1['has_discontinued_ops'] = (b1['do'] != 0).astype(int)
b1['has_extraordinary_items'] = (b1['xi'] != 0).astype(int) #not disclosed after 2016

Investment and Growth Opportunities

In [67]:
b1['investments'] = (b1['ivao'] + b1['ivst'] + b1['ivaeq']) / b1['at']
b1['capex'] = b1['capx'] / b1['at']
b1['rd_at'] = b1['xrd'] / b1['at']
b1['has_rd'] = (b1['xrd'] > 0).astype(int)
b1['has_merger'] = (b1['aqp'] != 0).astype(int)

Bankruptcy and Distress Indicators

In [52]:
b1['alt83_zscore'] = b1.apply(lambda row: 0.717*row['working_capital'] + 0.847*row['re']/row['at'] + 3.107*row['ebit']/row['at'] + 0.420*row['market_value']/row['dltt'] + 0.998*row['sale_at']
                             if pd.notnull(row['working_capital']) and pd.notnull(row['re']) and pd.notnull(row['ebit']) and pd.notnull(row['market_value']) and pd.notnull(row['sale_at']) and row['dltt'] != 0
                             else np.nan, axis=1)

Other common compustat variables

In [56]:
b1['has_foreign'] = (b1['fca'] != 0).astype(int)
b1['is_busy_season'] = (b1['month'] == 12).astype(int)

Specific Industry Indicators

In [57]:
litigious = (list(range(2833,2837)) + list(range(3570,3578)) + list(range(3600,3675)) + list(range(5200,5962)) + list(range(7370,7375))+list(range(8731,8735)))
utility = list(range(4400,4901))
financial = list(range(6000,7000))

#following Haislip et al. (2021) and based on Francis and Schipper (1999)
high_tech = [283, 357] + list(range(360, 369)) + [481, 737, 873]
high_tech2 = list(range(3600,3699)) + [4813, 7370, 8731]

In [58]:
b1['is_litigious'] = b1['sic'].apply(lambda x: 1 if x in litigious else 0)
b1['is_utility'] = b1['sic'].apply(lambda x: 1 if x in utility else 0)
b1['is_financial'] = b1['sic'].apply(lambda x: 1 if x in financial else 0)
b1['is_high_tech'] = b1['sic3'].apply(lambda x: 1 if x in high_tech else 0)
b1['is_high_tech2'] = b1['sic3'].apply(lambda x: 1 if x in high_tech2 else 0)

FF12 Industry Classification

In [59]:
def ff12(sic):
    if pd.isna(sic):
        return None

    sic_categories = {
        1: [(100, 1000), (2000, 2400), (2700, 2750), (2770, 2800), (3100, 3200), (3940, 3990)],
        2: [(2500, 2520), (2590, 2600), (3630, 3660), (3710,3712), 3714, 3716, (3750, 3752), 3972, (3900, 3940), (3990, 4000)],
        3: [(2520, 2590), (2600, 2700), (2750, 2770), (3000, 3100), (3200, 3570), (3580, 3630), (3700, 3710), (3712, 3714), 3715, (3717, 3750), (3752, 3792), (3793, 3800), (3830, 3840), (3860, 3900)],
        4: [(1200, 1400), (2900, 3000)],
        5: [(2800, 2830), (2840, 2900)],
        6: [(3570, 3580), (3660, 3693), (3694, 3700), (3810, 3830), (7370, 7380)],
        7: [(4800, 4900)],
        8: [(4900, 4950)],
        9: [(5000, 6000), (7200, 7300), (7600, 7700)],
        10: [(2830, 2840), 3693, (3840, 3860), (8000, 8100)],
        11: [(6000, 7000)],
        12: []
    }

    for category, ranges in sic_categories.items():
        for r in ranges:
            if isinstance(r, int):
                if sic == r:
                    return category
            elif isinstance(r,tuple):
                if r[0] <= sic < r[1]:
                    return category

    return 12

b1['ff12'] = b1['sic'].apply(ff12)

Create lagged variables

In [68]:
needs_lag = ['gvkey','fyear','datadate','at','size','roa','cfo','leverage', 'market_value','ib','oancf','sale','ppent','ceq','revt','intan','emp','prcc_f','invt','rd_at','sga','txr','ni','rect','roa']

In [69]:
b1 = b1.sort_values(by=['gvkey', 'datadate'])

is_consecutive = b1.groupby('gvkey')['fyear'].diff().eq(1)

for var in needs_lag:
    b1[f'lag_{var}'] = b1.groupby('gvkey')[var].shift(1)
    b1.loc[~is_consecutive, f'lag_{var}'] = np.nan

In [71]:
#check lags

cols_to_view = (
    ['gvkey', 'fyear', 'datadate'] +
    needs_lag +
    [f'lag_{v}' for v in needs_lag]
)

b1[cols_to_view].tail(20)

,gvkey,fyear,datadate,gvkey,fyear,datadate,at,size,roa,cfo,...,lag_intan,lag_emp,lag_prcc_f,lag_invt,lag_rd_at,lag_sga,lag_txr,lag_ni,lag_rect,lag_roa
212358,353945,2023,2023-12-31,353945,2023,2023-12-31,1206.801,7.095728,0.083864,None,...,10.540,2.500,20.693,186.287,0.052047,0.257116,14.914,122.801,230.889,0.125109
212359,353945,2024,2024-12-31,353945,2024,2024-12-31,1233.756,7.117818,0.137870,None,...,82.739,2.800,31.201,251.244,0.047604,0.273177,10.851,101.207,252.046,0.083864
212360,355398,2022,2022-12-31,355398,2022,2022-12-31,576.112,6.356302,-0.071597,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212361,355398,2023,2023-12-31,355398,2023,2023-12-31,576.883,6.357639,0.044468,None,...,3.059,NaN,NaN,25.984,0.000000,0.119300,0.000,-41.248,9.958,-0.071597
212362,355398,2024,2024-12-31,355398,2024,2024-12-31,684.425,6.528579,0.031853,None,...,0.402,12.891,NaN,29.762,0.000000,0.102254,0.000,25.653,18.430,0.044468
212363,356128,2022,2022-12-31,356128,2022,2022-12-31,11072.847,9.312251,0.114226,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212364,356128,2023,2023-12-31,356128,2023,2023-12-31,15046.521,9.618902,0.123330,None,...,33.085,7.802,66.000,0.000,0.011873,0.118605,0.000,1264.809,6820.605,0.114226
212365,356128,2024,2024-12-31,356128,2024,2024-12-31,15967.927,9.678337,0.124117,None,...,144.898,8.772,92.500,0.000,0.012996,0.101559,0.000,1855.692,9342.869,0.123330
212366,356289,2023,2023-06-30,356289,2023,2023-06-30,24.970,3.217675,0.089788,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212367,356289,2024,2024-06-30,356289,2024,2024-06-30,39.226,3.669340,-0.088564,None,...,0.000,0.015,NaN,0.449,NaN,0.389445,0.000,2.242,11.243,0.089788


Create additional variables that require lagged data

In [73]:
#changes in income before extraordinary items
b1['ib_chg'] = (b1['ib'] - b1['lag_ib']) / b1['lag_at']

#changes in roa
b1['roa_chg'] = b1['roa'] - b1['lag_roa']

#alternative ROA measure scaled by lagged assets
b1['roa_alt'] = b1['ib'] / b1['lag_at']

#cfo measure scaled by lagged assets
b1['cfo_alt'] = b1['oancf'] / b1['lag_at']

#sales growth
b1['sales_growth'] = (b1['sale'] - b1['lag_sale']) / b1['lag_sale']

#asset growth
b1['asset_growth'] = (b1['at'] - b1['lag_at']) / b1['lag_at']

#returns
b1['return'] = (b1['prcc_f'] - b1['lag_prcc_f']) / b1['lag_prcc_f']

Accruals Calculations

In [75]:
#total accruals (Chen et al. 2018; Aobdia 2019; Leuz et al. 2003)
b1['total_accruals'] = (b1['ib'] - (b1['oancf'] -  b1['xidoc'])) / b1['lag_at']
b1['abs_total_accruals'] = b1['total_accruals'].abs()

#total accruals relative to cash flows
b1['total_accruals_cfo'] = b1['total_accruals'] / ((b1['oancf'] - b1['xidoc']) / b1['lag_at'])
b1['abs_total_accruals_cfo'] = b1['total_accruals_cfo'].abs()

#total accruals scaled by average assets (Gipper et al. 2019 WP)
b1['avg_at'] = (b1['at'] + b1['lag_at']) / 2
b1['total_accruals_avg_at'] = (b1['ni'] - b1['oancf']) / b1['avg_at']
b1['abs_total_accruals_avg_at'] = b1['total_accruals_avg_at'].abs()

Extreme Sales Growth Indicator

In [76]:
b1['quintile'] = b1.groupby(['ff12','fyear'])['sales_growth'].transform(lambda x: pd.qcut(x.dropna(), q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=False, duplicates='drop'))
b1['extreme_sales_growth'] = b1['quintile'].apply(lambda x: 1 if x == 4 else 0)

Market Share

In [78]:
#Step 1: Calculate total industry sales by ff12 and year
industry_sales = b1.groupby(['ff12', 'fyear'])['sale'].sum().reset_index()
industry_sales = industry_sales.rename(columns={'sale': 'industry_sale'})

#Step 2: Merge industry sales back to main dataframe
b1 = b1.merge(industry_sales, on=['ff12', 'fyear'], how='left')

#Step 3: Calculate market share
b1['market_share'] = b1['sale'] / b1['industry_sale']

Create additional lags

In [80]:
needs_lag2 = ['total_accruals', 'roa_alt','cfo_alt']

b1 = b1.sort_values(by=['gvkey', 'datadate'])

is_consecutive = b1.groupby('gvkey')['fyear'].diff().eq(1)

for var in needs_lag2:
    b1[f'lag_{var}'] = b1.groupby('gvkey')[var].shift(1)
    b1.loc[~is_consecutive, f'lag_{var}'] = np.nan

Save final compustat file

In [81]:
b1.to_pickle(f"../Data/analysis/compustat_funda_final_{today}.pkl")